In [1]:
#!/usr/bin/env python3
# coding=utf-8
import pandas as pd
import numpy as np
import sqlite3
import time
from datetime import datetime, timedelta
import math
import sys
import logging
import os
# from scipy import stats as st
from scipy import stats 
from itertools import product as product
from matplotlib import pyplot as plt
# from sklearn.linear_model import LogisticRegression
import warnings
import statsmodels.api as sm
from math import sqrt
from tqdm import tqdm
import pymssql

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

# conn = pymssql.connect("127.0.0.1", "sa", "quant@123", "test_block")
# cursor = conn.cursor()

g_worthDir = '../pkl'  # 
g_parentDir = '../../..'  # 
g_dbfile = f"{g_parentDir}/pkl/hq.db" # 
g_name = g_access = 'sr_min'

g_locTimeadj = 28800 # 28800 0
g_version = '1.0.1'

print(f"{sys.version} , {sys.executable} , pandas: {pd.__version__}")
print(f"{time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()+g_locTimeadj))} *****import**v{g_version}*** ")

3.10.4 (main, Nov 29 2022, 20:00:25) [GCC 9.4.0] , /home/codespace/.python/current/bin/python3 , pandas: 1.5.2
2022-12-29 12:22:14 *****import**v1.0.1*** 


In [2]:
###日志 logger##20220825A######
def make_logger(name, log_level=logging.INFO, log_file="log.txt", file_mode="a"): #w写 a追加
    formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s:%(process)s: %(message)s')
    
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(level=log_level)
        handler = logging.FileHandler(log_file, mode=file_mode)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        handler = logging.StreamHandler(sys.stdout)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

logger = make_logger(g_name, log_level=logging.DEBUG, log_file= f"{g_parentDir}/logs/{g_name if ('g_name' in dir() ) else 'test' }_{ time.strftime('%Y%m%d') }.log" )

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****logging***** ')

2022-12-29 11:10:54 *****logging***** 


In [3]:
# ukdf.to_pickle(f"{g_worthDir}/uk5mdf.pkl" )
# ##df_uk = pd.read_pickle(f"{g_worthDir}/uk5mdf.pkl" )

# df_uk = pd.read_pickle(f"{g_parentDir}/pkl/uk5mdf.pkl" ) # uk5m_2021df 203451
df_uk = pd.read_pickle(f"/workspaces/jupyter/tmp/uk5m_2021df.pkl" ) # uk5m_2021df 203451
# df_uk.tail(2)
# print(f"{df_uk.iloc[-2:,:]}")
# display(df_uk.tail(2) )
print(f"df_uk.tail(2) :\n{df_uk.tail(2)}")

df_input = pd.DataFrame()
# df_input['date'] = df_uk['closeTime'].apply(lambda x: time.strftime("%Y%m%d",time.localtime(float(x)/1000) ) )
df_input['date'] = df_uk['tradeDate']
df_input['timestamp'] = df_uk['closeSec']
df_input['closeTime'] = df_uk['closeTime']
df_input['open'] = df_uk['open']
df_input['high'] = df_uk['high']
df_input['low'] = df_uk['low']
df_input['close'] = df_uk['close']
df_input['vol'] = df_uk['vol']
df_input['amount'] = df_uk['amt']
df_input['amt'] = df_uk['amt']/( df_uk['high']+ df_uk['low'])*2

# df_input.tail(2)
# display(df_input.tail(2) )
print(f"df_input.tail(2) :\n{df_input.tail(2)}")


df_uk.tail(2) :
       tradeDate openTime closeTime    closeSec     open    close     high  \
209227  20221228   113500    113959  1672198799  16824.8  16658.8  16824.8   
209228  20221228   114000    114459  1672199099  16652.5  16828.8  16828.8   

            low         vol           amt       pct  
209227  16647.0  468.325989  7.811034e+06  0.000228  
209228  16646.2  205.018997  3.424889e+06  0.010205  
df_input.tail(2) :
            date   timestamp closeTime     open     high      low    close  \
209227  20221228  1672198799    113959  16824.8  16824.8  16647.0  16658.8   
209228  20221228  1672199099    114459  16652.5  16828.8  16646.2  16828.8   

               vol        amount         amt  
209227  468.325989  7.811034e+06  466.723286  
209228  205.018997  3.424889e+06  204.623700  


In [4]:
def func(idx,df_input,thd):  # thd 0.8
    # thd = 3
    # print(f"{idx = }")
    idx = [int(i) for i in idx] # 144: 0~143
    ds = df_input.iloc[idx]
    thd = thd * 100
    ds['amt'] = ds['amt'] * (ds['high'] + ds['low']) / 2
    amt_sum = np.sum(ds['amt'])
    ds = ds[ds['indicator']>np.percentile(ds['indicator'],thd)]  # 分位数 80% 
    amt_pct = np.sum(ds['amt'])
    # ds['pct'] = np.sum(ds['amt']*(ds['high']+ds['low']))
    return amt_pct / amt_sum / (1-thd/100)


In [5]:
# loading_data('btc',wid_,thd,if_print,int(row_para[2]),row_para[3]) # 144.0 0.8 25.0 2.5 NaN  NaN
# index_para=9450 : wid_=192 : thd=0.88 : wid_back=10 : thd_std=4.0
wid_ = 192 
thd = 0.88
wid_back = 10 
thd_std = 4.0 

df_s = df_input.copy()

df_s['pct'] = df_s['close'] / df_s['close'].shift(1) - 1
df_s.fillna(0, inplace=True)
df_s['indicator'] = abs(df_s['pct']) /df_s['amt']
df_s['indicator_0'] = df_s['indicator']
df_s['index'] = df_s.index
df_s['value'] = df_s['index'].rolling(int(wid_)).apply(lambda x: func(x, df_s, thd)) # wid_ 144, thd 0.8

df_s['signal'] = np.nan
df_s['value_mean'] = df_s['value'].rolling(int(wid_*int(wid_back)) ).mean()  # wid_back 25 , 3600
df_s['value_std'] = df_s['value'].rolling(int(wid_*int(wid_back)) ).std()  # wid_back 25
df_s['signal'] = np.where((df_s['value'] > df_s['value_mean']+thd_std*df_s['value_std']), 1, df_s['signal'])   # thd_std 2.5
df_s['signal'] = np.where((df_s['value'] < df_s['value_mean']-thd_std*df_s['value_std']), -1, df_s['signal'])   # thd_std 2.5

print(f"df_s.tail(5) :\n{df_s.tail(5)}")

df_s.to_pickle(f"df_s.pkl" )
#df_s = pd.read_pickle(f"df_s.pkl" )

df_s.tail(5) :
            date   timestamp closeTime     open     high      low    close  \
209224  20221228  1672197899    112459  16650.0  16782.8  16650.0  16695.0   
209225  20221228  1672198199    112959  16755.0  16823.1  16627.2  16655.4   
209226  20221228  1672198499    113459  16654.7  16824.8  16627.2  16655.0   
209227  20221228  1672198799    113959  16824.8  16824.8  16647.0  16658.8   
209228  20221228  1672199099    114459  16652.5  16828.8  16646.2  16828.8   

               vol        amount         amt       pct     indicator  \
209224  366.703003  6.128018e+06  366.587207  0.002703  7.372605e-06   
209225  512.905029  8.575377e+06  512.723467 -0.002372  4.626212e-06   
209226  269.179993  4.492270e+06  268.580048 -0.000024  8.941928e-08   
209227  468.325989  7.811034e+06  466.723286  0.000228  4.888544e-07   
209228  205.018997  3.424889e+06  204.623700  0.010205  4.987114e-05   

         indicator_0   index     value  signal  value_mean  value_std  
209224  7.3

In [2]:
df_s = pd.read_pickle(f"/workspaces/jupyter/tmp/df_s_sr.pkl" ) 
# df_s2 = df_s[df_s['signal']>-2]
df_s2 = df_s[~df_s['signal'].isnull()]

print(f"{ df_s2.shape }")
print(f"{ df_s2.tail(2) }")

(2025, 18)
            date   timestamp closeTime     open     high      low    close  \
200591  20221128  1669607999    115959  16173.4  16184.0  16168.4  16184.0   
201132  20221130  1669770299    090459  16847.6  16999.0  16847.6  16921.7   

                vol        amount          amt       pct     indicator  \
200591   927.987000  1.501216e+07   928.039700  0.000556  5.995586e-07   
201132  1784.390015  3.017056e+07  1782.782168  0.003904  2.189655e-06   

         indicator_0   index     value  signal  value_mean  value_std  
200591  5.995586e-07  200591  1.481806     1.0    0.897727   0.136845  
201132  2.189655e-06  201132  1.545997     1.0    0.956857   0.146498  


In [ ]:
# nohup python3 -u main.py -n sr_min -s 8808 -c 29099 -X BTCUSDT -p 5m -w 192 -I 10 -d 20 -T 0.88 -H 4.0 -t 1000000 >> log.txt 2>&1 & 

# python3 -u main.py -n sr_min -s 8808 -c 29099 -X BTCUSDT -p 5m -w 192 -I 10 -d 20 -T 0.88 -H 4.0 -t 1000000 

In [3]:
if_print = 0
wid_ = np.arange(12)*4*12 + 144  # 12: [144 192 240 288 336 384 432 480 528 576 624 672]
thd = np.arange(31)*0.01 +0.6   # 31: [0.6  0.61 ... 0.88 0.89 0.9 ]
wid_back = np.arange(10)*5 +10  # 10: [10 15 20 25 30 35 40 45 50 55]
thd_std = np.arange(16)*0.2 + 2 # 16: [2. 2.2 2.4 2.6 2.8 3. 3.2 3.4 3.6 3.8 4. 4.2 4.4 4.6 4.8 5.]
df_para = pd.DataFrame(list(product(wid_, thd,wid_back,thd_std)))
df_para.columns = ['wid_','thd','wid_back','thd_std'] # 59520

print(f"wid_ : {len(wid_)}: {wid_}") 
print(f"wid_back : {len(wid_back)}: {wid_back}")
print(f"thd : {len(thd)}: {thd}")
print(f"thd_std : {len(thd_std)}: {thd_std}")
print(f"df_para : {len(df_para)}: {df_para}")

df_para['re']=np.nan
df_para['std']=np.nan
# print(df_para)
for index_para,row_para in tqdm(df_para.iterrows(),total=df_para.shape[0]):

    if index_para!=9450:
        if_print = 1
        continue
    wid_,thd=int(row_para[0]),row_para[1]
    print(f"{index_para=} : {wid_=} : {thd=} : wid_back={int(row_para[2])} : thd_std={row_para[3]}") # index_para=9450 : wid_=192 : thd=0.88 : wid_back=10 : thd_std=4.0

wid_ : 12: [144 192 240 288 336 384 432 480 528 576 624 672]
wid_back : 10: [10 15 20 25 30 35 40 45 50 55]
thd : 31: [0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73
 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87
 0.88 0.89 0.9 ]
thd_std : 16: [2.  2.2 2.4 2.6 2.8 3.  3.2 3.4 3.6 3.8 4.  4.2 4.4 4.6 4.8 5. ]
df_para : 59520:        wid_  thd  wid_back  thd_std
0       144  0.6        10      2.0
1       144  0.6        10      2.2
2       144  0.6        10      2.4
3       144  0.6        10      2.6
4       144  0.6        10      2.8
...     ...  ...       ...      ...
59515   672  0.9        55      4.2
59516   672  0.9        55      4.4
59517   672  0.9        55      4.6
59518   672  0.9        55      4.8
59519   672  0.9        55      5.0

[59520 rows x 4 columns]


 25%|██▌       | 14964/59520 [00:00<00:01, 31944.63it/s]

index_para=9450 : wid_=192 : thd=0.88 : wid_back=10 : thd_std=4.0


100%|██████████| 59520/59520 [00:02<00:00, 28975.37it/s]
